In [14]:
import numpy as np
import pandas as pd
from alpha_vantage.timeseries import TimeSeries
import copy 
import time
ts=TimeSeries(key='P9S7LN6VJ43Q6GAL',output_format='pandas')

In [4]:
def ATR(DF,n=14):
    df=DF.copy()
    df["H-L"] = df["High"]-df["Low"]
    df["H-PC"] = df["High"] - df["Adj Close"].shift(1)
    df["L-PC"] = df["Low"] - df["Adj Close"].shift(1)
    df["TR"] = df[["H-L","H-PC","L-PC"]].max(axis=1,skipna=False)
    df["ATR"]=df["TR"].ewm(com=n,min_periods=n).mean() #com gets you close to yahoo and span gets you close to traderview
    
    df2=df.drop(['H-L','H-PC','L-PC'],axis=1)
    return df2["ATR"]

In [11]:
def CAGR(DF):
    df = DF.copy()
    df["return"]=df["Adj Close"].pct_change()
    df["cumulative_return"]=(1+df["return"]).cumprod()
    n=len(df)/(252*78)   #Applying CAGR
    CAGR = (df["cumulative_return"].tolist()[-1])**(1/n)-1
    return CAGR

In [6]:
def sharpe(DF,rf):
    df=DF.copy()
    sr=(CAGR(df)-Risk_free_rate)/volatility(df)
    return sr

In [10]:
def max_dd(DF):
    df=DF.copy()
    df["cum_return"]=(1+df["return"]).cumprod()
    df["cum_roll_max"]=df["cum_return"].cummax()
    df["drawdown"]=df["cum_roll_max"]-df["cum_return"]
    max_dd=(df["drawdown"]/df["cum_roll_max"]).max()
    return max_dd

In [12]:
#Choose any tickers you want to test your backtesting on

In [13]:
tickers=["MSFT","AAPL","FB","AMZN","INTC","CSCO","VZ","IBM","TSLA","AMD"]

In [18]:
ohlc_intraday={}
api_call_count=1
ts=TimeSeries(key='P9S7LN6VJ43Q6GAL',output_format='pandas')
start_time=time.time()
for ticker in tickers:
    data=ts.get_intraday(symbol=ticker,interval='5min',outputsize='full')[0]
    api_call_count+=1
    data.columns=['Open','High','Low','Close','Volume']
    data = data.iloc[::-1] #unreverses reversed data given by alphavantage
    data=data.between_time('09:35','16:00')
    ohlc_intraday[ticker]=data
    if api_call_count==5:
                  api_call_count=1
                  time.sleep(60-((time.time()-start_time)%60.0))
tickers=ohlc_intraday.keys()

In [ ]:
#Data consolidation
ohlc_dict=copy.deepcopy(ohlc_intraday)
tickers_signal={}
ticker_ret={}
for ticker in tickers:
    print("calculating ATR and rolling max price for",ticker)
    ohlc_dict[ticker]["ATR"]=ATR(ohlc_dict[ticker],20)
    ohlc_dict[ticker]["roll_max_cp"]=ohlc_dict[ticker]["High"].rolling(20).max()
    ohlc_dict[ticker]["roll_min_cp"]=ohlc_dict[ticker]["Low"].rolling(20).min()
    ohlc_dict[ticker]["roll_max_vol"]=ohlc_dict[ticker]["Volume"].rolling(20).max()
    ohlc_dict[ticker].dropna(inplace=True)
    tickers_signal[ticker]= ""
    tickers_ret[ticker]=[]

In [ ]:
for ticker in tickers:
    print("calculating returns for",ticker)
    for i in range(len(ohlc_dict[ticker])):
        if tickers_signal[ticker]=="":
            tickers_ret[ticker].append(0)
            if ohlc_dict[ticker]["High"][i]>=ohlc_dict[ticker]["roll_max_cp"][i] and \
               ohlc_dict[ticker]["Volume"][i]>1.5*ohlc_dict[ticker]["roll_max_vol"][i-1]:
                tickers_signal[ticker]="Buy"
            elif ohlc_dict[ticker]["Low"][i]<=ohlc_dict[ticker]["roll_min_cp"][i] and \
              ohlc_dict[ticker]["Volume"][i]>1.5*ohlc_dict[ticker]["roll_max_vol"][i-1]:
                tickers_signal[ticker]="Sell"